In [1]:
import os 

In [2]:
%pwd

'/Users/jihaad/Documents/2023_Learning/Project/Text_Summarizer_Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/jihaad/Documents/2023_Learning/Project/Text_Summarizer_Project'